In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/my-private-datasets/quotes_rus/quotes_pattern01.txt
/kaggle/input/my-private-datasets/quotes_rus/quotes_pattern04.txt
/kaggle/input/my-private-datasets/quotes_rus/quotes_pattern03.txt
/kaggle/input/my-private-datasets/quotes_rus/quotes_pattern06.txt
/kaggle/input/my-private-datasets/quotes_rus/quotes_pattern05.txt
/kaggle/input/my-private-datasets/quotes_rus/quotes_pattern07.txt
/kaggle/input/my-private-datasets/quotes_rus/quotes_pattern02.txt
/kaggle/input/my-private-datasets/CVBooks/SoT_CV-Jr_2018_rus.txt
/kaggle/input/my-private-datasets/CVBooks/ollection_of_quotes_2015_rus.txt
/kaggle/input/my-private-datasets/CVBooks/rus/3rdAlt_2017.txt
/kaggle/input/my-private-datasets/CVBooks/rus/7h_1998.txt
/kaggle/input/my-private-datasets/CVBooks/rus/LonP_2008.txt
/kaggle/input/my-private-datasets/CVBooks/rus/7h_fam_2007.txt
/kaggle/input/my-private-datasets/CVBooks/rus/PredRes_2010.txt
/kaggle/input/my-private-datasets/CVBooks/rus/8h_2017.txt
/kaggle/input/my-private-datasets/C

- сравнить NER
- опробовать разные варианты dependencies
- можно делать это на саммаризованых текстах

In [2]:
SAMPLE_SIZE = 10000
pd.set_option('display.max_colwidth', -1)

df_sample = pd.read_csv("/kaggle/input/my-private-datasets/graph_news/news_with_summarization.csv", index_col=0).sample(SAMPLE_SIZE)
df_sample.rename(columns={"IlyaGusev__rut5_base_headline_gen_telegram": "summary"}, inplace=True)
df_sample

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,date,text,summary
153071,2022-05-21T15:21:27,"Вице-спикер Госдумы Чернышов предложил рассмотреть возможность введения дополнительного выходного за работу в условиях стресса.\n\nПрофсоюзы предложение поддержать готовы, но считают его реализуемость маловероятной, заявил РИА Новости глава ФНПР Шмаков.",В Госдуме предложили ввести дополнительный выходной за работу в условиях стресса
80701,2021-04-14T13:02:42,"Польская католическая церковь считает, что верующим можно прививаться AstraZeneca и Johnson & Johnson только при отсутствии альтернативы, поскольку их производство нарушает моральные принципы","Польская католическая церковь считает, что верующим можно прививаться AstraZeneca и Johnson & Johnson только при отсутствии альтернативы"
129459,2022-01-19T03:30:50,"Реклама в российском сегменте Google содержит тысячи вредоносных ссылок, часть из них ведет на поддельные сайты банков и их личные кабинеты, рассказали РИА Новости эксперты ОНФ",ОНФ: реклама в российском сегменте Google содержит тысячи вредоносных ссылок
75290,2021-03-17T12:21:24,"Тиньков дал совет, как вести хороший бизнес в России: ""бизнес всегда хороший, когда люди работают по 16 часов""","Тиньков: ""Бизнес всегда хороший, когда люди работают по 16 часов"""
48368,2020-10-12T18:08:42,"По уточненным данным, в стрельбе на остановке в Нижегородской области 3 погибли и 3 ранены, заявили РИА Новости в экстренных службах",В стрельбе на остановке в Нижегородской области погибли 3 человека
...,...,...,...
71914,2021-02-26T12:20:28,"Додон заявил, что советники президента Санду давят на правительственные органы, чтобы не допустить регистрации в Молдавии ""Спутник V""","Додон: Советники президента Молдавии давят на правительственные органы, чтобы не допустить регистрации ""Спутника V"""
141231,2022-03-09T16:14:57,Victoria's Secret останавливает работу магазинов и онлайн-продажи в России,Victoria's Secret останавливает работу магазинов и онлайн-продажи в России
533,2019-07-31T13:35:34,"❗️Twitter заблокировал аккаунт российского посольства в Сирии из-за критики ""Белых касок"".","Twitter заблокировал аккаунт посольства России в Сирии из-за критики ""Белых касок"""
136359,2022-02-22T19:29:28,"Путин: Европа не смогла заставить Киев выполнить минские договоренности, поэтому Россия была вынуждена принять решение о признании ДНР и ЛНР","Путин: Европа не смогла заставить Киев выполнить минские договоренности, поэтому Россия была вынуждена принять решение о признании ДНР и ЛНР"


In [3]:
# Clean ⚡️🎾❗️...
df_sample["text"] = df_sample.text.str.replace("[⚡️🎾❗️🌏]", "", regex=True)

## NER (compare spacy/stanza) ##

### Spacy ###

In [4]:
!python -m spacy download ru_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 667.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 8.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
tensorflow 2.6.4 requires h5py~=3.1.0, but you have h5py 3.7.0 which is incompatible.
tensorflow 2.6.4 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
tensorflow 2.6.4 requires tensorboard<2.7,>=2.6.0, but you have tensorboard 2.10.1 which is incompatible.
tensorflow 2.6.

In [5]:
import spacy
nlp_spacy = spacy.load("ru_core_news_lg")

In [6]:
timeit_sample = "Пересадки на наземном общественном транспорте в Москве станут бесплатными в течение 90 минут с начала поездки, объявил Собянин.До сих пор пересадка была бесплатной только между отдельными ветками метро (для сравнения, в некоторых городах мира за пересадку внутри метро надо платить) и между метро и поездами МЦК и МЦД."

In [7]:
%%timeit 
nlp_spacy(timeit_sample).ents

34.1 ms ± 1.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%%time
df_sample["NER_Spacy_Full"] = df_sample.text.map(lambda x: tuple(map(str, nlp_spacy(x).ents)))
df_sample["NER_Spacy_Summ"] = df_sample.summary.map(lambda x: tuple(map(str, nlp_spacy(x).ents)))

CPU times: user 4min 59s, sys: 275 ms, total: 5min
Wall time: 5min


### Stanza ###

In [9]:
!pip -q install stanza

In [10]:
import stanza
#nlp = stanza.Pipeline(lang='ru')
# nlp_stanza = stanza.Pipeline(lang='ru', processors='tokenize,pos,lemma,ner,depparse')
nlp_stanza = stanza.Pipeline(lang='ru', processors='tokenize,ner')

In [11]:
%%timeit 
nlp_stanza(timeit_sample).ents

422 ms ± 6.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%time
df_sample["NER_Stanza_Full"] = df_sample.text.map(lambda x: tuple(map(lambda y: y.text, 
                                                                      nlp_stanza(x).ents)))
df_sample["NER_Stanza_Summ"] = df_sample.summary.map(lambda x: tuple(map(lambda y: y.text, 
                                                                         nlp_stanza(x).ents)))

CPU times: user 1h 52min 11s, sys: 30.8 s, total: 1h 52min 42s
Wall time: 56min 30s


### Natasha

In [13]:
!pip -q install natasha

In [14]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

# text = 'Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав о решении властей Львовской области объявить 2019 год годом лидера запрещенной в России Организации украинских националистов (ОУН) Степана Бандеры. Свое заявление он разместил в Twitter. «Я не могу понять, как прославление тех, кто непосредственно принимал участие в ужасных антисемитских преступлениях, помогает бороться с антисемитизмом и ксенофобией. Украина не должна забывать о преступлениях, совершенных против украинских евреев, и никоим образом не отмечать их через почитание их исполнителей», — написал дипломат. 11 декабря Львовский областной совет принял решение провозгласить 2019 год в регионе годом Степана Бандеры в связи с празднованием 110-летия со дня рождения лидера ОУН (Бандера родился 1 января 1909 года). В июле аналогичное решение принял Житомирский областной совет. В начале месяца с предложением к президенту страны Петру Порошенко вернуть Бандере звание Героя Украины обратились депутаты Верховной Рады. Парламентарии уверены, что признание Бандеры национальным героем поможет в борьбе с подрывной деятельностью против Украины в информационном поле, а также остановит «распространение мифов, созданных российской пропагандой». Степан Бандера (1909-1959) был одним из лидеров Организации украинских националистов, выступающей за создание независимого государства на территориях с украиноязычным населением. В 2010 году в период президентства Виктора Ющенко Бандера был посмертно признан Героем Украины, однако впоследствии это решение было отменено судом. '
# doc = Doc(text)

In [15]:
def natasha_get_ents(sent):
    doc = Doc(sent)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger) # не обязательно
    doc.tag_ner(ner_tagger)
    return tuple([span.text for span in doc.spans])

In [16]:
%%time
df_sample["NER_Natasha_Full"] = df_sample.text.map(natasha_get_ents)
df_sample["NER_Natasha_Summ"] = df_sample.summary.map(natasha_get_ents)

CPU times: user 7min 55s, sys: 5min 28s, total: 13min 24s
Wall time: 3min 26s


### NER final compare

In [17]:
df_sample["NER_all_Full"] = (df_sample.NER_Spacy_Full + df_sample.NER_Stanza_Full + df_sample.NER_Natasha_Full).map(lambda x: tuple(set(x)))
df_sample["NER_all_Summ"] = (df_sample.NER_Spacy_Summ + df_sample.NER_Stanza_Summ + df_sample.NER_Natasha_Summ).map(lambda x: tuple(set(x)))

In [18]:
import itertools

list_dicts = []
for text_type in ["Full", "Summ"]:
    libs = ['Spacy', 'Stanza', 'Natasha']
    for lib in libs:
        df_sample[f"NER_missing_{lib}_{text_type}"] = df_sample.apply(lambda x: tuple(set(x[f"NER_all_{text_type}"]) - 
                                                                               set(x[f"NER_{lib}_{text_type}"])), 
                                                               axis=1)


    for a, b in list(itertools.combinations(range(3), 2)):
        df_sample[f"NER_missing_{libs[a]}{libs[b]}_{text_type}"] = df_sample.apply(lambda x: tuple(set(x[f"NER_all_{text_type}"]) - 
                                                                                      set(x[f"NER_{libs[a]}_{text_type}"]) -
                                                                                      set(x[f"NER_{libs[b]}_{text_type}"])), 
                                                                            axis=1)

    columns_NER_missing = [col_name for col_name in df_sample.columns if col_name.startswith("NER_missing_") and col_name.endswith(text_type)]
    dict_metrics = {}
    for col in columns_NER_missing:
        dict_metrics[col[:-5]] = {f"Accuracy ({text_type})": 1.0 - sum(df_sample[col].map(len)) / sum(df_sample[f"NER_all_{text_type}"].map(len)),
                                 f"Missing_count ({text_type})": sum(df_sample[col].map(len)),
                                 f"All_count ({text_type})": sum(df_sample[f"NER_all_{text_type}"].map(len))
                            }
    list_dicts.append(dict_metrics)
df_metrics = pd.concat(map(pd.DataFrame, list_dicts)).T
df_metrics

,Accuracy (Full),Missing_count (Full),All_count (Full),Accuracy (Summ),Missing_count (Summ),All_count (Summ)
NER_missing_Spacy,0.849578,4593.0,30534.0,0.860273,2685.0,19216.0
NER_missing_Stanza,0.830386,5179.0,30534.0,0.840602,3063.0,19216.0
NER_missing_Natasha,0.819873,5500.0,30534.0,0.799334,3856.0,19216.0
NER_missing_SpacyStanza,0.976125,729.0,30534.0,0.978039,422.0,19216.0
NER_missing_SpacyNatasha,0.898146,3110.0,30534.0,0.906172,1803.0,19216.0
NER_missing_StanzaNatasha,0.967937,979.0,30534.0,0.968047,614.0,19216.0


In [19]:
df_metrics.to_csv("metrics_ner_compare.csv")
df_metrics.to_excel("metrics_ner_compare.xlsx")

In [20]:
df_sample.to_csv("sample_ner_compare.csv")
df_sample.to_excel("sample_ner_compare.xlsx")

In [21]:
#df_sample = df_sample.apply(lambda column: column.map(lambda x: x if x != () else np.NaN))